In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
device

device(type='cuda')

In [1]:
from transformers import Trainer, TrainingArguments, AutoTokenizer #, GPT2Tokenizer, GPT2LMHeadModel
from datasets import load_dataset, load_from_disk
import os
from transformers import TextDataset, DataCollatorForLanguageModeling



# Load pre-trained GPT-2 model and tokenizer
# model_name = "gpt2"
# tokenizer = GPT2Tokenizer.from_pretrained(model_name)
# model = GPT2LMHeadModel.from_pretrained(model_name)

In [2]:
from llmsherpa.readers import LayoutPDFReader

llmsherpa_api_url = "https://readers.llmsherpa.com/api/document/developer/parseDocument?renderFormat=all"   

do_ocr = True
if do_ocr:
    llmsherpa_api_url = llmsherpa_api_url + "&applyOcr=yes"

pdf_reader = LayoutPDFReader(llmsherpa_api_url)
def extract_text_from_pdf(file_path):
    doc_obj = pdf_reader.read_pdf(file_path)
    text_data = ''
    for chunk in doc_obj.chunks():
        text_data += chunk.to_text()
    return text_data

In [45]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# tokenizer_flan = AutoTokenizer.from_pretrained("google/flan-t5-base")
# model_flan = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

# # Process PDF data from books and user manuals
# pdf_data_path = "./test_file"
# text_data = ""

# for filename in os.listdir(pdf_data_path):
#     if filename.endswith(".pdf"):
#         with open(os.path.join(pdf_data_path, filename), "rb") as file:
#             pdf_text = extract_text_from_pdf(file)
#             text_data += pdf_text

# # Tokenize the text data
# tokenized_text = tokenizer_flan(text_data, return_tensors="pt")

# # Create a TextDataset from the tokenized text
# dataset = TextDataset(tokenized_text, tokenizer=tokenizer_flan)

# # Define training arguments
# training_args = TrainingArguments(
#     output_dir="./output",
#     overwrite_output_dir=True,
#     num_train_epochs=3,
#     per_device_train_batch_size=4,
#     save_steps=10_000,
#     save_total_limit=2,
# )

# # Define Trainer for unsupervised fine-tuning
# trainer = Trainer(
#     model=model_flan,
#     args=training_args,
#     data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer_flan),
#     train_dataset=dataset,
# )

# # Perform unsupervised fine-tuning
trainer.train()

In [11]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from transformers import AutoTokenizer
import os
import PyPDF2


# 1. Text Extraction
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        text = ''
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text


In [2]:

pdf_data_path = './test_file'
text_data = ''
for filename in os.listdir(pdf_data_path):
    if filename.endswith('.pdf'):
        text_data += extract_text_from_pdf(os.path.join(pdf_data_path, filename))

# 2. Text Cleaning and Preprocessing
text_data = text_data.lower().strip()
text_data = ''.join(char for char in text_data if char.isalnum() or char.isspace())


In [16]:
# 4. Tokenization and Masking
tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-base', mask_token='[MASK]')
# tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [25]:
# # 3. Dataset Formatting
dataset = []
for i in range(0, len(text_data) - 100, 50):
    input_text = text_data[i:i+100]
    expected_output = text_data[i+100:i+150]
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    dataset.append({
        'input_ids': input_ids.squeeze(),
        'input_text': input_text,
        'expected_output': expected_output
    })


# # 3. Dataset Formatting
# dataset = []
# for i in range(0, len(text_data) - 100, 50):
#     input_text = text_data[i:i+100]
#     expected_output = text_data[i+100:i+150]
    
#     # Tokenize the input text
#     input_ids = tokenizer.encode(input_text, return_tensors='pt')
    
#     dataset.append({
#         'input_ids': input_ids.squeeze(),
#         'labels': tokenizer.encode(expected_output, return_tensors='pt').squeeze()
#     })


In [26]:

# 5. Load the Pre-Trained Model
model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-base')

In [27]:



# 6. Define the Fine-Tuning Setup
training_args = TrainingArguments(
    output_dir='./flan_t5_fine_tuned',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)




In [28]:

# 7. Perform Unsupervised Fine-Tuning
trainer.train()

  0%|          | 0/1530 [14:40<?, ?it/s]
                                                  
 33%|███▎      | 500/1530 [02:20<04:45,  3.61it/s]

{'loss': 6.5671, 'grad_norm': 10.753814697265625, 'learning_rate': 3.366013071895425e-05, 'epoch': 0.98}


                                                   
 65%|██████▌   | 1000/1530 [04:36<02:43,  3.24it/s]

{'loss': 6.0844, 'grad_norm': 9.513123512268066, 'learning_rate': 1.7320261437908496e-05, 'epoch': 1.96}


                                                   
 98%|█████████▊| 1500/1530 [06:53<00:09,  3.23it/s]

{'loss': 5.9277, 'grad_norm': 11.201403617858887, 'learning_rate': 9.80392156862745e-07, 'epoch': 2.94}


                                                   
100%|██████████| 1530/1530 [07:02<00:00,  3.62it/s]

{'train_runtime': 422.1127, 'train_samples_per_second': 14.491, 'train_steps_per_second': 3.625, 'train_loss': 6.190517620011872, 'epoch': 3.0}


TrainOutput(global_step=1530, training_loss=6.190517620011872, metrics={'train_runtime': 422.1127, 'train_samples_per_second': 14.491, 'train_steps_per_second': 3.625, 'total_flos': 263305251016704.0, 'train_loss': 6.190517620011872, 'epoch': 3.0})

In [36]:
# trainer.save_model('./flan_t5_fine_tuned')
trainer.model.save_pretrained("trained-model")

In [38]:
trainer.save_model()
tokenizer.save_pretrained('./flan_t5_tokenizer')


('./flan_t5_tokenizer\\tokenizer_config.json',
 './flan_t5_tokenizer\\special_tokens_map.json',
 './flan_t5_tokenizer\\spiece.model',
 './flan_t5_tokenizer\\added_tokens.json')

In [39]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the fine-tuned model
model = T5ForConditionalGeneration.from_pretrained('./flan_t5_fine_tuned')
tokenizer = T5Tokenizer.from_pretrained('./flan_t5_tokenizer')

# Example prompt for text generation
prompt = "what are some popular rag use cases?"

# Generate text based on the prompt
input_ids = tokenizer.encode(prompt, return_tensors='pt')
output = model.generate(input_ids, max_length=100, num_beams=4, early_stopping=True)

# Decode and print the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


rags can be used in a variety of ways ranging from rags to rags to rags to rags.


In [33]:
dataset

[{'input_ids': tensor([13339,    13,     8,   460,  2658,  2542,    30, 23941,  2254,    16,
            793,  1612,  3026,  1688,     3,  3916,  3301,  3264,     1]),
  'input_text': 'proceedings of the 2021 conference on empirical methods in natural language processing  pages 989599',
  'expected_output': '01\nnovember 711 2021 c\r2021 association for comput'},
 {'input_ids': tensor([    3,   189,    32,    26,     7,    16,   793,  1612,  3026,  1688,
              3,  3916,  3301,  3264,  4542,     3,  5326, 18247,   489,  2596,
            460,  2658,     3,    75,   460,  2658,  6028,    21,     3,   287,
           2562,     1]),
  'input_text': 'thods in natural language processing  pages 98959901\nnovember 711 2021 c\r2021 association for comput',
  'expected_output': 'ational linguistics9895picard \nparsing incremental'},
 {'input_ids': tensor([ 7088,     3,  5326, 18247,   489,  2596,   460,  2658,     3,    75,
            460,  2658,  6028,    21, 25850,     3, 24703,     

In [9]:
from transformers import AutoModelForSequenceClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

if os.path.exists('./google-bert_bert-base-cased'):
    model = AutoModelForSequenceClassification.from_pretrained('./google-bert_bert-base-cased')
else:
    model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased")
    model.save_pretrained('./google-bert_bert-base-cased')


In [23]:
modelQa = 'distilbert-base-cased-distilled-squad'

qa = pipeline('text-generation', model=model, tokenizer=tokenizer)

The model 'BertForSequenceClassification' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'Musicge

In [26]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("question-answering", model="google-bert/bert-large-uncased-whole-word-masking-finetuned-squad")

c:\Users\FUT_Novil\AppData\Local\anaconda3\envs\hfl\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\FUT_Novil\.cache\huggingface\hub\models--google-bert--bert-large-uncased-whole-word-masking-finetuned-squad. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at goo

In [44]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text2text-generation", model="google/flan-t5-base")

In [41]:
# pipe('What are the metrics use to measuer the performace of T5 + Picard Model') #[{'generated_text': 'metric'}]
# pipe('What is RAG?') # [{'generated_text': 'racial agglomeration'}]
pipe('what are the differences between RAG and Fine tuning?') #[{'generated_text': 'RAG is a tuning technique that uses a computer to perform a precise tuning of'}]

[{'generated_text': 'RAG is a tuning technique that uses a computer to perform a precise tuning of'}]